# TDoA-based Target Localization and Pursuit with Autonomous Vehicles: an Model Predictive Control Approach
The main aim of the work is to implement target pursuit with two surface trackers.

## Tracker Kinematic Model
We are going to use the simplest kinematic model of the tracker with only linear and angular velocities used as the input control.
$$
\begin{bmatrix}
\dot{x} \\
\dot{y} \\
\dot{\theta} \\
\end{bmatrix}
=
\begin{bmatrix}
v \cos{\theta} \\
v \sin{\theta} \\
\omega \\
\end{bmatrix}
=
\begin{bmatrix}
\cos{\theta} & 0 \\
\sin{\theta} & 0 \\
0 & 1 \\
\end{bmatrix}
\begin{bmatrix}
v \\
\omega
\end{bmatrix}
$$
where $\mathbf{x} = [x, y, \theta]^\intercal$ are position and angle of the tracker in the inertial reference frame (the **state** vector) and $\mathbf{u} = [v, w]^\intercal$ are the input of he system (the **control** vector).

## Target Model
The tracker moves along the predefined trajectory $\mathbf{q}(t) = [q_x(t), q_y(t)]^\intercal \in \mathbb{R}^2$.
The discrete version of the tracker trajectory is
\begin{equation}
    \mathbf{q}_k = [q_{x,k}, q_{y,k}]
\end{equation}

In [ ]:
# Load CasADi library
from casadi import *
import numpy as np

In [ ]:
# ---Define the discretization constants
DeltaT = 0.1 # Sampling time, c
PredictionN = 40 # Prediction horizon (prediction steps)

# ---Define the Control limitation
VLimit     = [0.0, 3.5]             # Limitation for vehicle linear velocity, m/s
OmegaLimit = [-np.pi / 2, np.pi / 2] # Limitation for vehicle angular velocity, rad/s

# Define Weight for running cost
Q = np.diag([1, 5, 0.1]) # Weight for the state vector, Matrix 3x3
R = np.diag([0.5, 0.05]) # Weight for the control vector, Matrix 2x2

# The initial state of the tracker
Tracker1StateInitial = np.array([[2.0], [2.0], [0]]) # Initial position

# Velocity of the target
TargetVelocity = 2.0

In [ ]:
from TrackerModel import TrackerModel

# Create MPC instance for the tracker 1
tracker1 = TrackerModel(label="tracker1", prediction_horizon=PredictionN)

# Create objective function
objective = 0
objective = objective + tracker1.create_objective(Q, R)

# Temporary define equality constraints as a python list
constraints_equality = tracker1.create_constraints(DeltaT)

ObstaclePosition = np.array([0.0, 0.0])
AvoidanceRadius = 1.0

# Add constraints connected with obstacle avoidance
# Just to check how it works
for step in range(0, PredictionN + 1):
    state = tracker1.get_state_prediction(step)
    constraints_equality.append(AvoidanceRadius - np.sqrt((state[0] - ObstaclePosition[0])**2 + (state[1] - ObstaclePosition[1])**2))

# Convert Python list to CasADi Matrix (vertical concatination)
constraints_equality = vertcat(*constraints_equality)

In [ ]:

st_size   = tracker1.state_vector.size()[0]
ctrl_size = tracker1.control_vector.size()[0]
        
# Reshape control variables (because we need to configure for NLP only one dimensional vector)
# But now we need to prepare the much larger vector of predictions of all states and controls within the receiving horizon
control_nlp = reshape(tracker1.get_state_prediction(), st_size * (PredictionN + 1), 1)
control_nlp = vertcat(control_nlp, reshape(tracker1.control_prediction, PredictionN * ctrl_size, 1))

# Configure the NLP solver (general view for all step within the prediction horizon)
nlp = {}
nlp['f'] = objective            # objective function
nlp['x'] = control_nlp          # Control input as column vector
nlp['p'] = vertcat(tracker1.state_init, tracker1.state_reference)              # the initial state and the target state
nlp['g'] = constraints_equality # the initial state and the target state

# Define the configuration options for the solver
opts = {}
opts["print_time"] = 0
opts["verbose"] = False
opts["ipopt.max_iter"] = 100
opts["ipopt.print_level"] = 0
opts["ipopt.acceptable_tol"] = 1e-8
opts["ipopt.acceptable_obj_change_tol"] = 1e-6

# Initialize the solver
solver = nlpsol('solver', 'ipopt', nlp, opts)

# Initialize the global constraints (see constraints_equality)
# To get equality condition we need to set upper and lower bound parameters as zeros
args = {}
# Equality constraints by kinematic model 
args["lbg"] = np.zeros((st_size * (PredictionN + 1) + PredictionN + 1, 1))
args["ubg"] = np.zeros((st_size * (PredictionN + 1) + PredictionN + 1, 1))

# Inequality constraints by obstacle avoidance
args["lbg"][st_size * (PredictionN + 1) : st_size * (PredictionN + 1) + PredictionN + 1] = -inf
args["ubg"][st_size * (PredictionN + 1) : st_size * (PredictionN + 1) + PredictionN + 1] = 0

# Set the lower bound for the optimized vector (both state and control vector)
args["lbx"] = np.zeros((st_size * (PredictionN + 1) + ctrl_size * PredictionN, 1))
# Fist let's fill the limit for state vector. First st_size * (PredictionN + 1) part
# (x, y parts)
args["lbx"][0:(3 * PredictionN + 1) + 0:3] = -np.inf
args["lbx"][1:(3 * PredictionN + 1) + 1:3] = -np.inf
# (\phi part)
args["lbx"][2:(3 * PredictionN + 1) + 2:3] = -np.inf
# Second let's fill the limit for control vector. Second ctrl_size * PredictionN part
args["lbx"][st_size * (PredictionN + 1) + 0:st_size * (PredictionN + 1) + ctrl_size * PredictionN:2] = np.min(VLimit)
args["lbx"][st_size * (PredictionN + 1) + 1:st_size * (PredictionN + 1) + ctrl_size * PredictionN:2] = np.min(OmegaLimit)

# Set the upper bound for the optimized vector (both state and control vector) symmetricaly
args["ubx"] = np.zeros((ctrl_size * PredictionN + st_size * (PredictionN + 1), 1))
args["ubx"][0:(3 * PredictionN + 1) + 0:3] = np.inf
args["ubx"][1:(3 * PredictionN + 1) + 1:3] = np.inf
args["ubx"][2:(3 * PredictionN + 1) + 2:3] = np.inf
args["ubx"][st_size * (PredictionN + 1) + 0:st_size * (PredictionN + 1) + ctrl_size * PredictionN:2] = np.max(VLimit)
args["ubx"][st_size * (PredictionN + 1) + 1:st_size * (PredictionN + 1) + ctrl_size * PredictionN:2] = np.max(OmegaLimit)

In [ ]:
from TrajectoryGenerator import TrajectoryGenerator
import matplotlib.pyplot as plt

# Control points for the target trajectory 
control_points = np.array([[  5.0,  5.0],
                           [  2.0,  15.],
                           [ 10.0,  18.],
                           [ 15.,   15.],
                           [ 20.,   25.],
                           [ 15.,   30.]
                           ])

# Create trajectory instance
trajectory = TrajectoryGenerator(control_points)

# Get all list of normalized parameters to build the target trajectory
parameter_list = np.linspace(0, 1, 100)
trajectory_points = trajectory.get_trajectory_point(parameter_list)

# Display trajectory of the target
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.set_title('The tracker trajectory on the plane')
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_xlim(0, 25)
ax.set_ylim(0, 35)
ax.grid(True, linestyle="--", alpha=0.7)
ax.minorticks_on()

ax.plot(trajectory_points[:, 0], trajectory_points[:, 1], '--', label='Target Trajectory', color='c', zorder=2)
ax.plot(control_points[:, 0], control_points[:, 1], '-o', label='Control Points', color='b', zorder=2)
ax.scatter(trajectory_points[0, 0], trajectory_points[0, 1], marker='o', s=50, label='Start', color='g', zorder=3)
ax.scatter(trajectory_points[-1, 0], trajectory_points[-1, 1], marker='o', s=50, label='Finish', color='r', zorder=3)
ax.legend(loc='upper left')

In [ ]:
def getTrajectoryPart(trajectory, parameter: float, steps: int):
    vector = []
    
    for st in range(0, steps):
        # check is this is the end of trajectory
        terminal_step = parameter > 1.0

        # Limit the trajectory parameter for predictions
        if terminal_step:
            parameter = 1.0

        # Get target coordinate
        target_point = trajectory.get_trajectory_point(parameter)

        # Calculate tangent vector at the point
        trajectory_derivative = trajectory.get_trajectory_derivative(parameter)
        target_yaw = np.arctan2(trajectory_derivative[1], trajectory_derivative[0])

        # Configure target state
        target_state = np.append(target_point, target_yaw)

        # Calculate a new parameter delta to preserve velocity constant
        parameter_delta_delta = TargetVelocity * DeltaT / np.linalg.norm(trajectory_derivative)
        parameter = parameter + parameter_delta_delta

        # Add state vector within the horizon
        vector.append(target_state)
        # Add control vector within the horizon (zero linear velocity if this is the terminal state)
        vector.append(np.array([TargetVelocity if not terminal_step else 0.0, 0.0]))
    return vector

In [ ]:
import time

# We don't have any prediction, so zeros are our best guess
control_guess = np.zeros((PredictionN, ctrl_size))
# The state on the first point will be not far of the Tracker1StateInitial
state_guess = repmat(Tracker1StateInitial, 1, PredictionN + 1)

tracker1.init_storages()

# Storage for target states
storage_target_states = []
        
# Current state of the robot. At the beginning it coincides with the x_start
tracker1_state_current = Tracker1StateInitial

# Start timer to check performance
time_start = time.process_time()

# Target trajectory parameter
parameter = 0

# Main Loop of the motion control
while parameter < 1.0:
    #----- Update target position
    # Get target coordinate
    target_point = trajectory.get_trajectory_point(parameter)

    # Calculate tangent vector at the point
    trajectory_derivative = trajectory.get_trajectory_derivative(parameter)
    target_yaw = np.arctan2(trajectory_derivative[1], trajectory_derivative[0])

    # Configure target state
    target_state = np.append(target_point, target_yaw).reshape((-1, 1))


    target_future_trajectory = getTrajectoryPart(trajectory, parameter, PredictionN)
    
    #----- Prepare initial vector for tracker 1
    # Upload new initial state (new each step) and the target state (for each point within the prediction horizon)
    # First part is current state
    initial_vector = []
    initial_vector.append(tracker1_state_current.flatten())
    initial_vector = initial_vector + target_future_trajectory

    # Reshape the initial vector to standard representation
    initial_vector = np.concatenate(initial_vector, axis=0)
    initial_vector = initial_vector.reshape(-1, 1)

    args["p"]  = initial_vector

    #----- Prepare initial guess for tracker 1
    # Set initial guess
    args["x0"] = reshape(state_guess, st_size * (PredictionN + 1), 1)
    args["x0"] = vertcat(args["x0"], reshape(control_guess.T, ctrl_size * PredictionN, 1))

    #----- Solve the OCP
    sol = solver(x0=args["x0"], p=args["p"],
                 lbx=args["lbx"], ubx=args["ubx"],
                 lbg=args["lbg"], ubg=args["ubg"]
                 )

    # Reshape obtained contol predictions vector to matrix (2xPredictionN)
    # Extract only control predictions
    control_predictions = reshape(sol['x'][st_size * (PredictionN + 1):].T, ctrl_size, PredictionN).T
    # We need only the first control value in the storage
    control = control_predictions[0, :]

    # Extract only state predictions
    state_predictions = reshape(sol['x'][:st_size * (PredictionN + 1)].T, st_size, PredictionN + 1)

    #----- Storage data for tracker 1
    # Store the state vector on the each step of simulation
    tracker1.storage_states.append(tracker1_state_current)

    # Store predicted states in the list of 2D Matrix
    tracker1.storage_predictions.append(state_predictions.T)

    # Store only the first control value in the storage
    tracker1.storage_control.append(control)

    # Store current target state
    storage_target_states.append(target_state)

    #----- Shift state of the tracker 1
    # Shift the time and initial position for solver
    # We propagate control actions to obtain the new state of the system at this point
    tracker1_state_current = tracker1_state_current + (DeltaT *  tracker1.kin_function(tracker1_state_current, control.T))
    tracker1_state_current = np.array(tracker1_state_current)

    # Calculate a new parameter delta for target motion to preserve velocity constant
    parameter_delta = TargetVelocity * DeltaT / np.linalg.norm(trajectory_derivative)
    parameter = parameter + parameter_delta

    #----- Shift state and control guess of the tracker 1
    # We the control action predictions and state predictions as the best guess for the next control
    # Just trim the first row, because we already used it and append zero row at the end
    control_guess = np.delete(control_predictions, 0, axis=0)
    control_guess = np.vstack((control_guess, control_predictions[-1, :]))

    state_guess = np.delete(state_predictions, 0, axis=1)
    state_guess = np.hstack((state_guess, state_predictions[:, -1]))

# Simulation steps number
simulation_steps = len(tracker1.storage_control)

elapsed_time = time.process_time() - time_start
print(f"Overal OCP time for all steps: {elapsed_time} sec. Per step: {elapsed_time / simulation_steps}")

tracker1.convert_storages()

# Concatinate all data along the second axis (because state is the column-vector)
storage_target_states = np.concatenate(storage_target_states, axis=1)

In [ ]:
import matplotlib.pyplot as plt
from IPython import display
import utility as ut
%matplotlib inline

fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.set_title('The vehicle movement on the plane')
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_xlim(0, 25)
ax.set_ylim(0, 35)
ax.grid(True, linestyle="--", alpha=0.7)

# Heading Figure size
markersize = 40

# Plot the target trajectory
ax.plot(trajectory_points[:, 0], trajectory_points[:, 1], '--', label='Target Trajectory', color='c', zorder=2)

for i in range(simulation_steps):
    marker_tracker, scale = ut.gen_arrow_head_marker(tracker1.storage_states[2, i])
    pointer_tracker = ax.scatter(tracker1.storage_states[0, i], tracker1.storage_states[1, i],
                                 marker=marker_tracker, s=(markersize*scale)**2, color='green', zorder=3)
    
    marker_target, scale = ut.gen_arrow_head_marker(storage_target_states[2, i])
    pointer_target = ax.scatter(storage_target_states[0, i], storage_target_states[1, i],
                                marker=marker_target, s=(markersize*scale)**2, color='red', zorder=3)
    
    trajectory_actual = ax.plot(tracker1.storage_states[0, 0:i], tracker1.storage_states[1, 0:i], 
                                linestyle="--", linewidth=1.5, color='black', 
                                label='Tracker Trajectory', zorder=1)[0]
    trajectory_predict = ax.scatter(tracker1.storage_predictions[:, 0, i], tracker1.storage_predictions[:, 1, i],
                                    color='purple', marker='x', s=10, label='Tracker Trajectory Predicted', zorder=1)
    ax.legend(loc='upper left')

    # Update plot rendering
    display.clear_output(wait=True)
    display.display(fig)

    # Wait to render the current step of the trajectory
    # with 5x faster than simulation process
    fast_forward_scale = 5
    time.sleep(DeltaT / fast_forward_scale)

    # Remove pointer in order to redraw it on the next stage
    trajectory_predict.remove()
    trajectory_actual.remove()
    pointer_tracker.remove()
    pointer_target.remove()

plt.close(fig)